In [1]:
# Improved_Ionosphere_regression file is modified from Ionosphere_regression.ipynb to improve the performance

# Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
%matplotlib inline

from torch import nn, optim                           # torch 에서 제공하는 신경망 기술, 손실함수, 최적화를 할 수 있는 함수들을 불러온다.
from torch.utils.data import DataLoader, Dataset      # 데이터를 모델에 사용할 수 있게 정리해주는 라이브러리.

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Loss
from sklearn.metrics import mean_squared_error        # regression 문제의 모델 성능 측정을 위해서 MSE를 불러온다.

In [2]:
# Read Data
dir_path = "C:/Python/Deep_learning/Ionosphere-TEC-regression-with-pytorch/"

train = pd.read_csv( dir_path + "Preprocessed_data/datafile_nan.csv")

# Index column drop
# Index 열은 Quality에 영향을 주지 않음
train = train.drop(['xq'], axis=1)

# 데이터 타입에 따라 분류
numerical_columns = train.select_dtypes(exclude='object').columns.tolist()

# 확인을 위한 호출
print(numerical_columns)

['Kp_inter', 'Ap_inter', 'F107_inter', 'SunSpot_inter', 'dst', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'T13', 'T14', 'T15', 'T16', 'C_D', 'S_D', 'C_H', 'S_H', 'tec_ex(T1)']


In [3]:
Y = train['tec_ex(T1)'].values
X = []

for i, rows in train.iterrows():
    # 데이터 프레임을 가로 한줄씩 출력 row
    X.append([ rows['Kp_inter'], rows['Ap_inter'], rows['F107_inter'], rows['SunSpot_inter']
                , rows['T1'], rows['T2'], rows['T3'], rows['T4'], rows['T5'], rows['T6'], rows['T7'], rows['T8'], rows['T9']
                , rows['T10'], rows['T11'], rows['T12'], rows['T13'], rows['T14'], rows['T15'], rows['T16']
                , rows['C_D'], rows['S_D'], rows['C_H'], rows['S_H']])

    # remove 'dst' column, because it has low correlation with 'tec_ex(T1)'

In [4]:
Y = Y.reshape((-1,1)) # reshape(-1,1) 열 값은 지정했으니 알아서 행 지정해서 배열로 만들기

# 데이터 스케일링
# sklearn에서 제공하는 MinMaxScaler 
# (X-min(X))/(max(X)-min(X))을 계산
scalerX = MinMaxScaler()
scalerX.fit(X)
X = scalerX.transform(X)

scalerY = MinMaxScaler()
scalerY.fit(Y)
Y = scalerY.transform(Y)

In [ ]:
# Hyper parameter tuning

CFG = {
    'EPOCHS':20, #에포크
    'LEARNING_RATE':3e-4, #학습률
    'BATCH_SIZE':16, #배치사이즈
    'SEED':41, #시드
}

In [ ]:
# torch의 Dataset 을 상속.
class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
# 전체 데이터를 학습 데이터와 평가 데이터로 나눈다.
X_train, X_T, Y_train, Y_T = train_test_split(X, Y, test_size=0.4)
X_val, X_test, Y_val, Y_test = train_test_split(X_T, Y_T, test_size=0.5)
# train : val : test = 0.6 : 0.2 : 0.2

# 학습 데이터, 시험 데이터 배치 형태로 구축하기
trainsets = TensorData(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=CFG['BATCH_SIZE'], shuffle=True)

valsets = TensorData(X_val, Y_val)
valloader = torch.utils.data.DataLoader(valsets, batch_size=CFG['BATCH_SIZE'], shuffle=False)

testsets = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(testsets, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [ ]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(25, 64, bias=False),
            nn.BatchNorm1d(64, eps=1e-05, momentum=0.1),
            nn.ReLU()
        )
        
        self.layer2 = nn.Sequential(
            nn.Linear(64, 128, bias=False),
            nn.BatchNorm1d(128, eps=1e-05, momentum=0.1),
            nn.ReLU()
        )
        
        self.layer3 = nn.Sequential(
            nn.Linear(128, 256, bias=False),
            nn.BatchNorm1d(256, eps=1e-05, momentum=0.1),
            nn.ReLU()
        )
        
        self.layer4 = nn.Linear(256, 1, bias=False)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
      
        return x

In [ ]:
model = Regressor()
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'], weight_decay=1e-7)

In [ ]:
loss_ = [] # loss 저장할 리스트
rmse_ = [] # val rmse 저장할 리스트

def train(model, optimizer, trainloader, valloader):
    n = len(trainloader)

    # Loss Function
    criterion = nn.MSELoss()
    best_acc = 0

    for epoch in range(1, CFG["EPOCHS"]+1):
        model.train()
        running_loss = 0.0

        for i, data in enumerate(trainloader, 0):
            inputs, values = data

            optimizer.zero_grad() # 최적화 초기화

            outputs = model(inputs) # 예측값 산출
            loss = criterion(outputs, values) # Error 계산

            loss.backward() # 역전파 진행
            optimizer.step() # 역전파 진행 후 가중치 업데이트

            running_loss += loss.item() # Epoch 마다 평균 loss를 계산하기 위한 배치 loss
                                        # item() 텐서로 값 받아오기

        loss_.append(running_loss/n) # MSE 계산

        # print('[%d] Train loss: %.10f' %(epoch, running_loss / len(trainloader)))

        #validation set 평가

        model.eval() #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
        val_loss = 0.0

        predictions = torch.tensor([], dtype=torch.float) # 예측값을 저장하는 텐서.
        actual = torch.tensor([], dtype=torch.float) # 실제값을 저장하는 텐서.

        with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
            for i, data in enumerate(valloader, 0): # enumerate(인자, index)
                inputs, values = data
                outputs = model(inputs)

                predictions = torch.cat((predictions, outputs), 0) # cat함수를 통해 예측값을 누적.
                actual = torch.cat((actual, values), 0) # cat함수를 통해 실제값을 누적.

                # if i == len(valloader)-1:
                #     torch.save(model.state_dict(), dir_path + "last_model.pth")

        predictions = predictions.numpy() # 넘파이 배열로 변경.
        actual = actual.numpy() # 넘파이 배열로 변경.
        rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용해 RMSE를 계산.
        rmse_.append(rmse)

        # print(f'val rmse:{rmse}')



In [ ]:
from sklearn.model_selection import KFold

# split 개수, shuffle, seed 설정
kf = KFold(n_splits=5, shuffle=True, random_state=41)
n_iter = 0

# for train_index, test_index in str_kf.split(X, Y):
# 이거는 Lable도 동일한 비율로 나눌 수 있어서 좋음
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]

    train(model, optimizer, trainloader, valloader)

    n_iter += 1
    torch.save(model.state_dict(), dir_path + str(n_iter) +"last_model.pth")
    print('{} 번째, 학습데이터 크기 : {}, 검증데이터 크기 : {}'.format(n_iter, X_train.shape[0], X_val.shape[0]))

In [ ]:
plt.clf()
# plt.plot(loss_,'ro',label='training loss')
plt.plot(rmse_,'g',label='validation loss')
plt.title('Training and validation accuracy')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
# test set 평가

def predict(model, testloader):
    model.eval()  #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
    model_pred = []
    test_loss = 0.0

    with torch.no_grad():  #파라미터 업데이트 안하기 때문에 no_grad 사용
        for i, data in enumerate(testloader, 0):
            inputs, values = data
            outputs = model(inputs)
            test_loss += criterion(outputs, values)

            model_pred.extend(outputs.tolist())
        print(f'test loss:{test_loss}')
    return model_pred

In [ ]:
# load last model

for i in range(5):
    checkpoint = torch.load(dir_path + str(i+1) + "last_model.pth")
    model = Regressor()
    model.load_state_dict(checkpoint)


    if i == 0:
        preds1 = predict(model, testloader)
        preds1 = scalerY.inverse_transform(preds1)
    if i == 1:
        preds2 = predict(model, testloader)
        preds2 = scalerY.inverse_transform(preds2)
    if i == 2:
        preds3 = predict(model, testloader)
        preds3 = scalerY.inverse_transform(preds3)
    if i == 3:
        preds4 = predict(model, testloader)
        preds4 = scalerY.inverse_transform(preds4)
    if i == 4:
        preds5 = predict(model, testloader)
        preds5 = scalerY.inverse_transform(preds5)

pred = (preds1 + preds2 + preds3 + preds4 + preds5)/5



In [ ]:
Y_test_rescale = scalerY.inverse_transform(Y_test)

pred[5:10], Y_test_rescale[5:10]

In [ ]:
submission = pd.read_csv( dir_path + "sample_submission.csv")
submission['tec_ex(T1)'] = pred

submission.to_csv( dir_path + "submit.csv", index=False)